# Update Tables

This notebook exists as a way to track changes made to the tables in nfl_gamethreads.db.
#TODO: Update create_tables.ipynb with these features for future proofing and replication.

### Imports

In [1]:
# Path hack
import os
# change directory from the current Analysis folder to the top level folder for easier navigation
os.chdir('../')
# confirm we're at /RedditTextAnalysis
print(os.getcwd())

/Users/prcork/MiscDataProjects/collaboration/RedditTextAnalysis


In [2]:
# note that the %load_ext autoreload line only needs to be be run once
%load_ext autoreload

In [3]:
# by including this autoreload command, we only need to re-import if we make/save changes to the original py file
%autoreload
from Functions.nfl_gamethreads import nfl_gamethreads

In [4]:
import pandas as pd
import sqlite3
import re

### Query Comments Table

In [5]:
with sqlite3.connect("nfl_gamethreads.db") as conn:

    # pull in all submission ids that have previously been crawled
    comments_df = pd.read_sql_query("SELECT * FROM comments", conn)

In [6]:
comments_df

,comment_id,submission_id,author,body,upvotes,utc_time,author_flair
0,hejgunm,pwto53,drumline17,Lebron's high school QB has probably been tell...,169,1.632792e+09,:Broncos: Broncos
1,hej6lhu,pwto53,Mr_Alex19,Friday Night Lights vs It's Always Sunny in Ph...,163,1.632787e+09,:Rams: Rams
2,hejhqt0,pwto53,flagg-deschain,I didn’t think anything could be worse than ge...,160,1.632792e+09,:Eagles: Eagles
3,hejbb99,pwto53,None,[deleted],298,1.632789e+09,None
4,heja73s,pwto53,usersince2012,"""If you're gonna get a personal foul, get it o...",415,1.632789e+09,:49ers: 49ers
...,...,...,...,...,...,...,...
772575,hlkus6s,qz47b2,bigDean636,It's almost unwatchable except the tv ratings ...,1,1.637541e+09,:Chiefs: Chiefs
772576,hlkvmqy,qz47b2,None,[removed],-1,1.637542e+09,None
772577,hlllv1i,qz47b2,Commercial-Yak-8450,Ok my fault we were missing the number 2 ranke...,1,1.637554e+09,None
772578,hlm6a8i,qz47b2,None,[deleted],1,1.637567e+09,None


## 1. Process Author Flairs

This process transforms author flairs from ":team: team" to "team".
This featured was introduced in the enhanced get_comments() function, but was not in the original table. Thus this process queries all pre-existing comments, processes the author flair, and updates each observation in the dataframe, replacing the raw author_flair data.

In [7]:
comments_df['author_flair'] = comments_df['author_flair'].apply(lambda flair: nfl_gamethreads.process_flair(flair))

In [11]:
# confirm there are only 34 options (32 teams plus 'nfl' plus 'none')
len(comments_df['author_flair'].unique().tolist())

34

In [12]:
# save to SQL db, overwriting raw data (which is also saved in /Data as a backup measure)
comments_df.to_sql(name='comments', con=conn, if_exists='replace', index=False)

772580

## 2. Add Polarity and Subjectivity Fields to Comments DB by Default

The original batch of comments were queried and stored under the assumption that text analysis would happen dynamically in exploratory analysis. However, the time consuming nature of this given the volume of comments suggests it best to complete this process at ingestion. Thus, two updates must be made.

1. The comments table needs two float columns added, one for polarity and one for subjectivity
2. Similar to above, the entire set of pre-existing comments needs to be run through analyze_text() and those new columns stored in the database.

In [14]:
with sqlite3.connect("nfl_gamethreads.db") as conn:
    c = conn.cursor()

    add_pol = '''ALTER TABLE comments
    ADD COLUMN polarity real;'''

    add_sub = '''ALTER TABLE comments
    ADD COLUMN subjectivity real;'''

    c.execute(add_pol)
    c.execute(add_sub)

    # pull in all submission ids that have previously been crawled
    comments_df = pd.read_sql_query("SELECT * FROM comments", conn)

In [16]:
comments_df_ta = nfl_gamethreads.analyze_text(comments_df, 'body')

In [18]:
comments_df_ta.to_sql(name='comments', con=conn, if_exists='replace', index=False)

772580